In [1]:
from pathlib import Path
import xarray as xr
import numpy as np
import tiledb
import pandas as pd
from datetime import datetime
import time
import tifffile
import matplotlib.pyplot as plt

In [2]:
SATELLITE_DATA_PATH = Path('~/data/EUMETSAT/reprojected/just_UK/2019/06/03/').expanduser()
TIFF_PATH = Path('~/data/EUMETSAT/tiff/reprojected/just_UK/2019/06/03/').expanduser()

In [3]:
if not TIFF_PATH.exists():
    TIFF_PATH.mkdir(parents=True)

In [4]:
netcdf_filenames = SATELLITE_DATA_PATH.glob('*.nc')
netcdf_filenames = list(netcdf_filenames)
netcdf_filenames.sort()
num_nc_files = len(netcdf_filenames)
print(f'{num_nc_files} netcdf files found.')

18 netcdf files found.


In [36]:
# Write TIFFs

def data_array_to_tiff(da):
    for dt in da.time.values:
        ts = pd.Timestamp(dt)
        raw_image = da.sel(time=ts)
        # normalise TODO: Use hard-coded number to normalise by!
        image = raw_image.copy().values
        image /= image.max()
        image *= 255
        #image = image.astype(np.float16)
        image = image.round().astype(np.uint8)
        filename = ts.strftime('HRV_%Y-%m-%dT%H%M.tiff')
        filename = TIFF_PATH / filename
        with tifffile.TiffWriter(filename) as tiff:
            tiff.save(image, compress=9)

In [37]:
%%time

for filename in netcdf_filenames:
    da = xr.load_dataset(filename)['HRV']
    data_array_to_tiff(da)

CPU times: user 43.2 s, sys: 2.44 s, total: 45.6 s
Wall time: 46.5 s


In [38]:
tiff_filenames = TIFF_PATH.glob('*.tiff')
tiff_filenames = list(tiff_filenames)
tiff_filenames.sort()
len(tiff_filenames)

216

In [39]:
%%time
for i, filename in enumerate(tiff_filenames):
    _ = tifffile.imread(filename)

CPU times: user 3.05 s, sys: 448 ms, total: 3.5 s
Wall time: 1.19 s


In [40]:
%%time
_ = tifffile.imread(tiff_filenames)

CPU times: user 3.12 s, sys: 436 ms, total: 3.56 s
Wall time: 1.25 s


In [41]:
%%time
for i, filename in enumerate(tiff_filenames):
    img = xr.open_rasterio(filename)
    _ = img[0, 128:256, 128:256]

/Users/JackKelly/miniconda3/envs/storage_benchmarks/lib/python3.8/site-packages/rasterio/__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


CPU times: user 340 ms, sys: 108 ms, total: 448 ms
Wall time: 451 ms


float32
* all: 4.58 s
* rectangle: 0.529 s



float16
* all: 2.57s
* rect: 0.383s

uint8
* all 1.19s
* rect 0.453s